In [38]:
import requests
import re

MISTRAL_API_KEY = "e6ojRsfwNwYFzpVQjpHtX8YavnOrIjcI"
MODEL = "mistral-small-latest"
API_URL = "https://api.mistral.ai/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {MISTRAL_API_KEY}",
    "Content-Type": "application/json"
}

def check_credibility(text):
    prompt = f"""
You are a misinformation detection expert.

Analyze the following online content for credibility:
\"\"\"{text}\"\"\"

Step 1: Identify if the text contains misleading, exaggerated, or false claims.
Step 2: Detect patterns typical of misinformation.
Step 3: Output a credibility score from 0 (false) to 100 (credible) and a reason.

Respond in this format only:
Credibility Score: <score>
Reason: <brief reason>
"""
    body = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }

    response = requests.post(API_URL, headers=headers, json=body)
    if response.status_code != 200:
        raise Exception(f"❌ API Error: {response.status_code} - {response.text}")

    content = response.json()['choices'][0]['message']['content']
    match = re.search(r"Credibility Score: (\d+)\s*Reason: (.*)", content, re.DOTALL)
    
    if match:
        score = int(match.group(1))
        reason = match.group(2).strip()
        return score, reason
    else:
        return None, "Couldn't parse response."

# === TEST ===
if __name__ == "__main__":
    #text = "hannah montana is a real person"
    text = input("Enter a statement")
    try:
        score, reason = check_credibility(text)
        print(f"\n🔎 Score: {score}/100")
        print(f"📝 Reason: {reason}")
    except Exception as e:
        print(e)


Enter a statement newton is a famous scientist and it is also the name of a cookie



🔎 Score: 100/100
📝 Reason: The statement is factual and verifiable. Isaac Newton is indeed a famous scientist, and "Newton" is also a well-known brand of cookies. There are no misleading, exaggerated, or false claims present.


In [33]:
import requests
import re
import json
from time import sleep

MISTRAL_API_KEY = "e6ojRsfwNwYFzpVQjpHtX8YavnOrIjcI"
MODEL = "mistral-small-latest"
API_URL = "https://api.mistral.ai/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {MISTRAL_API_KEY}",
    "Content-Type": "application/json"
}

def get_mistral_score(claim):
    prompt = f"""
You are a misinformation detection expert.

Analyze the following online content for credibility:
\"\"\"{claim}\"\"\"

Step 1: Identify if the text contains misleading, exaggerated, or false claims.
Step 2: Detect patterns typical of misinformation.
Step 3: Output a credibility score from 0 (false) to 100 (credible) and a reason.

Respond in this format only:
Credibility Score: <score>
Reason: <brief reason>
"""
    body = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7
    }

    response = requests.post(API_URL, headers=headers, json=body)
    if response.status_code != 200:
        print(f"⚠️ API Error: {response.status_code} - {response.text}")
        return None, "API error"

    content = response.json()['choices'][0]['message']['content']
    match = re.search(r"Credibility Score: (\d+)\s*Reason: (.*)", content, re.DOTALL)

    if match:
        score = int(match.group(1))
        reason = match.group(2).strip()
        return score, reason
    else:
        return None, "Couldn't parse response"

def load_sample_fever_data(path='path/to/fever/train.jsonl', limit=10):
    samples = []
    try:
        with open(path, 'r') as f:
            for i, line in enumerate(f):
                if i >= limit:
                    break
                entry = json.loads(line)
                samples.append(entry)
        return samples
    except FileNotFoundError:
        print(f"❌ File not found: {path}")
        print("👉 Please check the dataset path and try again.")
        return []

# === MAIN PIPELINE ===
if __name__ == "__main__":
    fever_data = load_sample_fever_data(path='train.jsonl', limit=10)
    results = []

    for idx, item in enumerate(fever_data):
        claim = item['claim']
        label = item['label']

        print(f"\n🔍 Checking claim #{idx+1}: {claim}")
        score, reason = get_mistral_score(claim)

        print(f"✅ Label: {label}")
        print(f"🧠 Mistral Score: {score}")
        print(f"✍️ Reason: {reason}")

        results.append({
            "claim": claim,
            "label": label,
            "mistral_score": score,
            "reason": reason
        })

        sleep(1)  # Sleep to avoid API rate limits

    with open("mistral_fever_results.json", "w") as f:
        json.dump(results, f, indent=2)
        print("\n📁 Saved results to 'mistral_fever_results.json'")



🔍 Checking claim #1: Nikolaj Coster-Waldau worked with the Fox Broadcasting Company.
✅ Label: SUPPORTS
🧠 Mistral Score: 80
✍️ Reason: The claim is straightforward and does not contain obvious exaggerations or falsehoods. Nikolaj Coster-Waldau has indeed worked with various production companies, and it is plausible that he has worked with Fox Broadcasting Company. However, without specific details or verification, the claim is moderately credible.

🔍 Checking claim #2: Roman Atwood is a content creator.
✅ Label: SUPPORTS
🧠 Mistral Score: 95
✍️ Reason: The statement "Roman Atwood is a content creator" is a factual claim that can be easily verified. Roman Atwood is a well-known YouTuber and content creator, so the statement is credible. There are no misleading, exaggerated, or false claims present, and no patterns typical of misinformation are detected.

🔍 Checking claim #3: History of art includes architecture, dance, sculpture, music, painting, poetry literature, theatre, narrative, fi